In [1]:
using JuMP

┌ Info: Recompiling stale cache file C:\Users\vicml\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190


In [2]:
using GLPK

┌ Info: Recompiling stale cache file C:\Users\vicml\.julia\compiled\v1.0\GLPK\r6CoY.ji for GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1190


### Questão 7

<h4><b>The company C & O operates an oil pipeline pumping oil from Alberta to
various states in the Northwestern USA. Figure 1.2 shows the direction of
flow, four input lines, and the three output lines. Note for instance that State
A can only get its oil from either Input 1 or from the Yukon input line.<b><h4>

                                        Yukon  Input1   Input2   Input3
                                         |       |        |         |
                                        ---------------------------------->
                                                      |        |        |
                                                   StateA   StateB   StateC

<h4><b>Each input line has a capacity (barrels/day) and a cost per barrel:<b><h4>

| Input Lines         | 1    | 2    | 3    | Yukon |
|---------------------|------|------|------|-------|
| Capacity            | 4000 | 2000 | 3000 | 10000 |
| Cost per barril ($) | 70   | 50   | 30   | 60    |

<h4><b>Each state has a daily demand (barrels/day) that must be met exactly:<h4><b>

| States | A    | B    | C    |
|--------|------|------|------|
| Demand | 3500 | 3000 | 4000 |

<h4><b>The input from the Yukon is not owned by the company and activating that
line has a fixed cost of $11 000 per day.
Write an IP that plans the activities of the company C & O for a day (how
many barrels of oil to pump from each input line) by minimizing the total
daily cost of the company while meeting all the demand.<b><h4>

### Resolução

Primeiro criamos as variáveis de custo, limite e capacidade do problema

In [18]:
demA = 3500
demB = 3000
demC = 4000

lim1 = 4000
lim2 = 2000
lim3 = 3000
limy = 10000

custob1 = 70
custob2 = 50
custob3 = 30
custoby = 60
custoy = 11000

11000

Para modelar esse problema, criamos as variáveis $x_1,x_2,x_3,x_4$ inteiras correspondentes a quantidade de barris inputada pelas linhas 1,2,3 e de Yukon respectivamente. Além disso, temos a possibilidade de utilizar ou não a linha de Yukon, por isso, acrescentamos também uma variável binária y que assumirá valor 1 quando a pista de Yukon for utilizada ($x_4 > 0$) e valor 0 caso contrário.

Sabemos que queremos minimizar o custo total da distribuição, então nossa função objetivo é:

$
min \qquad custob1 \cdot x_1 + custob2 \cdot x_2 + custob3 \cdot x_3 + custob4 \cdot x_4 + custoy \cdot y
$

Para as restrições de limite, temos que cada linha tem um máximo de barris de capacidade, portanto temos que:

$
x_1 \leq lim1\\
x_2 \leq lim2\\
x_3 \leq lim3\\
x_4 \leq lim4
$

Além disso, nos é informado que cada cidade tem que receber exatamente uma quantidade especificada de barris, o que nos leva a entender que quando o óleoduto passa por cada cidade i, é preciso que se tenha disponível uma quantidade de barris maior ou igual a demanda da cidade i. Dessa forma, temos que:

$
x_1 + x_4 \geq demA \\
(x_1+x_4-demA) + x_2 \geq demB\\
((x_1+x_4-demA)+x_2-demB)+x_3 \geq demC
$

Agora, temos que pensar em uma maneira de modelar a variável y de forma que ela fique dependente do valor de $x_4$. Para isso utilizamos o valor de $\frac{x_4}{limy}$ como limite inferior de y, pois essa métrica é 0 quando o valor de $x_4$ é 0 e assume um valor maior que 0 quando $x_4$ assume um valor maior que 0. Dessa forma, como temos definido que y é uma variável binária, ela assumirá o valor de 0 quando não tivermos barris vindos de Yukon e 1 caso contrário.

$
y \geq \frac{x_4}{limy}
$

### Aplicando o Solver em Julia

In [19]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,x[1:4]>=0)
@variable(model,y,Bin)

# Restrições
@constraint(model,demanda_de_A,x[4] + x[1] >= demA)
@constraint(model,demanda_de_B,(x[4] + x[1]-demA)+x[2]>=demB)
@constraint(model,demanda_de_C,((x[4] + x[1]-demA)+x[2]-demB)+x[3]>=demC)
@constraint(model,limite_yukon,x[4]<=limy)
@constraint(model,limite_1,x[1]<=lim1)
@constraint(model,limite_2,x[2]<=lim2)
@constraint(model,limite_3,x[3]<=lim3)
@constraint(model,limite_y,y>=x[4]/limy)

# Função objetivo
@objective(model,Min,custob1*x[1] + custob2*x[2] + custob3*x[3] + custoby*x[4] + custoy*y)

print(model)

Min 70 x[1] + 50 x[2] + 30 x[3] + 60 x[4] + 11000 y
Subject to
 demanda_de_A : x[1] + x[4] >= 3500.0
 demanda_de_B : x[1] + x[2] + x[4] >= 6500.0
 demanda_de_C : x[1] + x[2] + x[3] + x[4] >= 10500.0
 limite_y : -0.0001 x[4] + y >= 0.0
 limite_yukon : x[4] <= 10000.0
 limite_1 : x[1] <= 4000.0
 limite_2 : x[2] <= 2000.0
 limite_3 : x[3] <= 3000.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 y binary


In [20]:
optimize!(model)

In [21]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [22]:
objective_value(model)

531000.0

In [23]:
value.(x)

4-element Array{Float64,1}:
    0.0
 2000.0
 3000.0
 5500.0

In [24]:
value(y)

1.0

### RESPOSTA FINAL:

                                        Yukon  Input1           Input2         Input3
                                       (5500)   (0)             (2000)         (3000)
                                         |       |       (2000)    |   (1000)    |
                                        --------------------------------------------------->
                                                      |              |                 |
                                                   StateA          StateB            StateC
                                                   (3500)          (3000)            (4000)